In [1]:
%load_ext autoreload
%autoreload 2

from copy import copy
import numpy as np
import parmed as pmd
import mdtraj as mdt
import subprocess
from pathlib import Path
from parmed.topologyobjects import CmapType, Cmap
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append("../../")
# from data_utils import build_cmap_atoms
from add_cmap_to_gmx import add_cmap_to_topol, add_cmap_to_pmd_top, build_cmaps

sys.path.append("../../analysis/")
# from chem_shift import run_faspr
from add_sidechains import add_sidechains
from cmap import build_phipsi


In [3]:

# basepath = "/data/gzappavigna/lmp_conf_builder/gmx/amber/"
# basepath = "./a99-ildn/"
ref_basepath = "/home/gzappavigna/traj/20241018_ff99sb-ildn_opc_multidir/"
# name = "step5_prodution"
name = "final"

# ref_traj = mdt.iterload(basepath + "step5_production_nojump.xtc", top=basepath + "step5_production_nojump.pdb")
ref_traj = mdt.load(ref_basepath + f"{name}_nojump.xtc", top=ref_basepath + f"{name}_nojump.pdb", stride=10)


In [4]:
# ens_basepath = "../"
# ens_traj = mdt.load(ens_basepath + "ensemble.xtc", top=ens_basepath + "ensemble.pdb")

In [5]:
ens_path = "../../MeV_NT_ens"
cmd = rf'find {ens_path}/ensemble_200_* -regex "^.*/[0-9]+a_132\.pdb$"'
res = subprocess.run(cmd, shell=True, capture_output=True, text=True)

# pdbs = [Path(pdb) for pdb in res.stdout.splitlines()]
# pdbs_all = add_sidechains(pdbs)

pdbs = [mdt.load(pdb) for pdb in res.stdout.splitlines()]

In [6]:
ens_phipsi = np.concatenate(
    [build_phipsi(frame) for frame in pdbs], axis=1
)

ref_phipsi = build_phipsi (ref_traj)
# ref_phipsi = np.concatenate(
#     [build_phipsi(frame) for frame in ref_trajs], axis=1
# )

cmaps = build_cmaps(ens_phipsi, ref_phipsi)


vol: 0.991836588690999
vol: 0.988207068643131
vol: 0.9897830597329522
vol: 0.9943455336229733
vol: 0.9898704259289008
vol: 0.9752848911316164
vol: 0.9833410018335659
vol: 0.9840330071815686
vol: 0.9894595314294345
vol: 0.9827896444606048
vol: 0.9875319524357402
vol: 0.9893318293436748
vol: 0.9925928810100619
vol: 0.9872514900460576
vol: 0.9952543340089182
vol: 0.9959185410448527
vol: 0.9956942992517019
vol: 0.9992440444021321
vol: 0.9909419259404902
vol: 0.9937428417536696
vol: 0.9974695804053827
vol: 0.995559740555862
vol: 0.9993810056192638
vol: 0.9987596455829637
vol: 0.994359579107447
vol: 0.9993619378080228
vol: 0.9936902920872726
vol: 0.9969094802938794
vol: 0.9910718830878744
vol: 0.9967840505896469
vol: 0.9936583661891423
vol: 0.9947749917618158
vol: 0.9307988271858849
vol: 0.9098533267970521
vol: 0.9807983163677805
vol: 0.9985045438034534
vol: 0.9929593474778757
vol: 0.999385130926037
vol: 0.9907740442153163
vol: 0.9963593149546454
vol: 0.992080562606089
vol: 0.992285954220606

In [7]:
add_cmap_to_topol(ref_basepath + "topol.top", ref_basepath + "ntail_solv_ions.gro", pdbs, ref_traj)

FileNotFoundError: /home/gzappavigna/traj/20241018_ff99sb-ildn_opc_multidir/topol.top does not exist

In [4]:
ref_phipsi = build_phipsi(ref_traj)

if isinstance(ens_traj, mdt.Trajectory):
    ens_phipsi = build_phipsi(ens_traj)
else:
    ens_phipsi = np.concatenate([build_phipsi(frame) for frame in ens_traj], axis=1)

pmd_top = pmd.load_file(topol_file, xyz=xyz_file)
cmap_pmd_top = add_cmap_to_pmd_top(pmd_top, ens_phipsi, ref_phipsi)


NameError: name 'ref_traj' is not defined